## Multi Layer Perceptron

In [15]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import Image
%matplotlib inline

In [16]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
n_train = x_train.shape[0]
n_test = x_test.shape[0]

We scale the data to be in $[-1,1]$.

In [17]:
x_train, x_test = x_train / 127.5 - 1, x_test / 127.5 - 1

The size of each data points is $28 \times 28$. Vectorize the datapoints for visualization and preliminary questions.

In [18]:
nb_features = np.prod(x_train.shape[1:])
x_train.resize((n_train, nb_features))
x_test.resize((n_test, nb_features))

In [19]:
print(x_train.shape)

(60000, 784)


### 3: Multi Layer Perceptron 

Multi Layer Perceptron (MLP) is a fully connected deep (more than one hidden layer) network. In this part, implement a 2 hidden layers MLP with Recified Linear Unit (ReLU) activations. We will train the model via ADAM optimizer over a cross-entropy loss function.

First of all, we will convert our label vectors to matrices via one-hot encoding (e.g. $y=2$ would become $[0,0,1,0,0,0,0,0,0,0]$). This can be simply done using commands below:

In [12]:
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

Below, we define a class MLP. It is initialized via:  

+ x_train: The training matrix.
+ y_train: One hot encoding of the corresponding labels.
+ lr: Learning rate used for ADAM optimizer
+ nb_epochs: Number of epochs to use
+ batch_size: The number of data point in each mini-batch
+ output_dir: The directory where model parameters and tensorboard event files will be stored.

We also define the methods: 
* 'create_model' which will desribe a neural network architecture of the form $[784, 1000, 1000, 10]$, each integer representing the number of neurons in a given layer while the length of the vector defines the number of layers accordingly. 
* 'compute_loss' which given the output of 'create_model' will calculate the cross-entropy loss of the mini-batches.
* 'train' where we initiate a tensorflow session and perform the training iterations. 
* 'test' where we load our trained model and perform inference on the test set.

##### Task 3.1
- Complete the method 'create_model' in order to implement a network of the shape $[784, 1000, 1000, 10]$, use ReLU as the non linear activation for hidden layers.

   The function 'create_model' to complete defines the class variables: 

   + self.logits $\in \mathbb{R^{10}}$ containing the output __<font color='red'>without activation of the MLP.</font>__
   + self.preds $\in \mathbb{R^{10}}$ containing posterior probabilities.

- Using self.logits complete the method 'compute_loss' that takes the labels and the predicted logits to return the corresponfing cross-entropy loss. 

In [13]:
class MLP:
    def __init__(self, x_train, y_train, output_dir, lr=0.001, nb_epochs=10, batch_size=50):
        tf.reset_default_graph()
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.batch_size = batch_size
        self.nb_epochs = nb_epochs
        self.nb_images, self.nb_features = x_train.shape
        self.nb_iterations = self.nb_images // batch_size
        self.output_dir = output_dir
        self.im = tf.placeholder(tf.float32, [None, 784])
        self.labels = tf.placeholder(tf.float32, [None, 10])
        self.x_train = x_train
        self.y_train = y_train
        
    def create_model(self):
        with tf.variable_scope('MLP', reuse=tf.AUTO_REUSE):
        ######### Complete the function ######### 
            hl_1 = tf.layers.dense(self.im,1000,activation=tf.nn.relu, name='hl_1')
            hl_2 = tf.layers.dense(hl_1,1000,activation=tf.nn.relu, name='hl_2')
            self.logits = tf.layers.dense(hl_2,10,activation=None, name='logits')
            self.preds = tf.nn.softmax(self.logits, name='preds')
    def compute_loss(self):
        with tf.variable_scope('loss'):
            
            self.loss = tf.losses.softmax_cross_entropy(self.labels, self.logits)
            self.loss_summ = tf.summary.scalar("softmax_loss", self.loss)
            
    def optimizer(self):
        with tf.variable_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.lr, beta1=0.5)
            self.model_vars = tf.trainable_variables()
            self.trainer = optimizer.minimize(self.loss, var_list=self.model_vars)

Now that we defined our model, our loss and its optimizer. we can instantate the MLP class, initiate our variables, and start the tensorflow session:

In [14]:
model = MLP(x_train, y_train, './MLP_logdir/', 0.001, 2, 10)
model.create_model()
model.compute_loss()
model.optimizer()
init = (tf.global_variables_initializer(),
        tf.local_variables_initializer())

saver = tf.train.Saver()
summary =tf.Summary()
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.summary.FileWriter(model.output_dir)
writer.add_graph(sess.graph)
if not os.path.exists(model.output_dir):
    os.makedirs(model.output_dir)


We can now start training. We loop over the training data points and we feed them to the session in mini-batches form. we repeat this process several times (for several epochs). 

In [15]:
for epoch in range(model.nb_epochs):
    randomize = np.arange(x_train.shape[0])
    np.random.shuffle(randomize)
    x_in = model.x_train[randomize,:]
    y_in = model.y_train[randomize,:]
    for i in range(model.nb_iterations):
        input_x_train = x_in[i*model.batch_size: (i+1)*model.batch_size]
        input_y_train = y_in[i*model.batch_size: (i+1)*model.batch_size]
        _ , preds, loss, loss_summ = sess.run([model.trainer, model.preds, model.loss, model.loss_summ], 
                                 feed_dict={model.im: input_x_train, 
                                            model.labels: input_y_train})
        y_preds = np.argmax(preds, axis=1)
        y_real = np.argmax(input_y_train, axis=1)
        acc_train = np.mean((y_preds==y_real)*1)
        print('Epoch %d, Iteration %d, loss %.3f, batch accuracy %.3f' %(epoch, i, loss, acc_train))
        writer.add_summary(loss_summ, epoch * model.nb_iterations + i)
    saver.save(sess, model.output_dir, global_step=epoch)  
# sess.close()

Epoch 0, Iteration 0, loss 2.671, batch accuracy 0.200
Epoch 0, Iteration 1, loss 3.334, batch accuracy 0.100
Epoch 0, Iteration 2, loss 4.824, batch accuracy 0.100
Epoch 0, Iteration 3, loss 3.680, batch accuracy 0.200
Epoch 0, Iteration 4, loss 1.459, batch accuracy 0.500
Epoch 0, Iteration 5, loss 1.797, batch accuracy 0.300
Epoch 0, Iteration 6, loss 2.297, batch accuracy 0.400
Epoch 0, Iteration 7, loss 2.809, batch accuracy 0.200
Epoch 0, Iteration 8, loss 1.453, batch accuracy 0.700
Epoch 0, Iteration 9, loss 2.753, batch accuracy 0.100
Epoch 0, Iteration 10, loss 2.002, batch accuracy 0.200
Epoch 0, Iteration 11, loss 1.880, batch accuracy 0.300
Epoch 0, Iteration 12, loss 1.688, batch accuracy 0.400
Epoch 0, Iteration 13, loss 1.561, batch accuracy 0.500
Epoch 0, Iteration 14, loss 1.379, batch accuracy 0.600
Epoch 0, Iteration 15, loss 1.636, batch accuracy 0.500
Epoch 0, Iteration 16, loss 1.147, batch accuracy 0.600
Epoch 0, Iteration 17, loss 1.693, batch accuracy 0.100
Ep

Epoch 0, Iteration 166, loss 0.539, batch accuracy 0.800
Epoch 0, Iteration 167, loss 0.918, batch accuracy 0.600
Epoch 0, Iteration 168, loss 0.937, batch accuracy 0.700
Epoch 0, Iteration 169, loss 0.248, batch accuracy 0.900
Epoch 0, Iteration 170, loss 2.103, batch accuracy 0.400
Epoch 0, Iteration 171, loss 0.636, batch accuracy 0.800
Epoch 0, Iteration 172, loss 1.911, batch accuracy 0.500
Epoch 0, Iteration 173, loss 1.909, batch accuracy 0.500
Epoch 0, Iteration 174, loss 1.034, batch accuracy 0.800
Epoch 0, Iteration 175, loss 0.395, batch accuracy 0.900
Epoch 0, Iteration 176, loss 0.435, batch accuracy 0.900
Epoch 0, Iteration 177, loss 0.769, batch accuracy 0.800
Epoch 0, Iteration 178, loss 1.139, batch accuracy 0.400
Epoch 0, Iteration 179, loss 0.598, batch accuracy 0.800
Epoch 0, Iteration 180, loss 0.526, batch accuracy 0.600
Epoch 0, Iteration 181, loss 0.471, batch accuracy 0.800
Epoch 0, Iteration 182, loss 0.060, batch accuracy 1.000
Epoch 0, Iteration 183, loss 0.

Epoch 0, Iteration 317, loss 0.324, batch accuracy 0.900
Epoch 0, Iteration 318, loss 0.647, batch accuracy 0.700
Epoch 0, Iteration 319, loss 1.243, batch accuracy 0.400
Epoch 0, Iteration 320, loss 0.704, batch accuracy 0.700
Epoch 0, Iteration 321, loss 0.841, batch accuracy 0.700
Epoch 0, Iteration 322, loss 0.628, batch accuracy 0.700
Epoch 0, Iteration 323, loss 0.334, batch accuracy 0.800
Epoch 0, Iteration 324, loss 0.158, batch accuracy 1.000
Epoch 0, Iteration 325, loss 0.136, batch accuracy 1.000
Epoch 0, Iteration 326, loss 0.640, batch accuracy 0.800
Epoch 0, Iteration 327, loss 0.363, batch accuracy 0.800
Epoch 0, Iteration 328, loss 0.406, batch accuracy 0.900
Epoch 0, Iteration 329, loss 0.133, batch accuracy 1.000
Epoch 0, Iteration 330, loss 0.536, batch accuracy 0.900
Epoch 0, Iteration 331, loss 0.976, batch accuracy 0.800
Epoch 0, Iteration 332, loss 1.263, batch accuracy 0.700
Epoch 0, Iteration 333, loss 0.455, batch accuracy 0.900
Epoch 0, Iteration 334, loss 0.

Epoch 0, Iteration 477, loss 0.992, batch accuracy 0.800
Epoch 0, Iteration 478, loss 0.683, batch accuracy 0.800
Epoch 0, Iteration 479, loss 0.061, batch accuracy 1.000
Epoch 0, Iteration 480, loss 0.298, batch accuracy 0.900
Epoch 0, Iteration 481, loss 0.270, batch accuracy 0.800
Epoch 0, Iteration 482, loss 0.211, batch accuracy 0.900
Epoch 0, Iteration 483, loss 0.216, batch accuracy 0.900
Epoch 0, Iteration 484, loss 0.038, batch accuracy 1.000
Epoch 0, Iteration 485, loss 0.641, batch accuracy 0.700
Epoch 0, Iteration 486, loss 0.259, batch accuracy 0.900
Epoch 0, Iteration 487, loss 0.405, batch accuracy 0.800
Epoch 0, Iteration 488, loss 0.122, batch accuracy 1.000
Epoch 0, Iteration 489, loss 0.264, batch accuracy 1.000
Epoch 0, Iteration 490, loss 0.066, batch accuracy 1.000
Epoch 0, Iteration 491, loss 0.995, batch accuracy 0.800
Epoch 0, Iteration 492, loss 0.551, batch accuracy 0.700
Epoch 0, Iteration 493, loss 0.220, batch accuracy 0.900
Epoch 0, Iteration 494, loss 0.

Epoch 0, Iteration 660, loss 0.230, batch accuracy 0.900
Epoch 0, Iteration 661, loss 0.519, batch accuracy 0.900
Epoch 0, Iteration 662, loss 0.232, batch accuracy 0.900
Epoch 0, Iteration 663, loss 0.305, batch accuracy 0.900
Epoch 0, Iteration 664, loss 0.117, batch accuracy 1.000
Epoch 0, Iteration 665, loss 0.050, batch accuracy 1.000
Epoch 0, Iteration 666, loss 0.231, batch accuracy 1.000
Epoch 0, Iteration 667, loss 0.364, batch accuracy 0.900
Epoch 0, Iteration 668, loss 0.737, batch accuracy 0.700
Epoch 0, Iteration 669, loss 0.045, batch accuracy 1.000
Epoch 0, Iteration 670, loss 0.057, batch accuracy 1.000
Epoch 0, Iteration 671, loss 0.433, batch accuracy 0.900
Epoch 0, Iteration 672, loss 0.348, batch accuracy 0.900
Epoch 0, Iteration 673, loss 0.684, batch accuracy 0.600
Epoch 0, Iteration 674, loss 0.048, batch accuracy 1.000
Epoch 0, Iteration 675, loss 0.909, batch accuracy 0.700
Epoch 0, Iteration 676, loss 0.127, batch accuracy 1.000
Epoch 0, Iteration 677, loss 0.

...

Epoch 1, Iteration 771, loss 0.000, batch accuracy 1.000
Epoch 1, Iteration 772, loss 0.591, batch accuracy 0.900
Epoch 1, Iteration 773, loss 0.492, batch accuracy 0.900
Epoch 1, Iteration 774, loss 0.009, batch accuracy 1.000
Epoch 1, Iteration 775, loss 0.070, batch accuracy 1.000
Epoch 1, Iteration 776, loss 0.067, batch accuracy 1.000
Epoch 1, Iteration 777, loss 0.136, batch accuracy 0.900
Epoch 1, Iteration 778, loss 0.392, batch accuracy 0.900
Epoch 1, Iteration 779, loss 0.138, batch accuracy 1.000
Epoch 1, Iteration 780, loss 0.224, batch accuracy 0.900
Epoch 1, Iteration 781, loss 0.492, batch accuracy 0.700
Epoch 1, Iteration 782, loss 0.004, batch accuracy 1.000
Epoch 1, Iteration 783, loss 0.012, batch accuracy 1.000
Epoch 1, Iteration 784, loss 0.056, batch accuracy 1.000
Epoch 1, Iteration 785, loss 0.332, batch accuracy 0.900
Epoch 1, Iteration 786, loss 0.101, batch accuracy 1.000
Epoch 1, Iteration 787, loss 0.069, batch accuracy 1.000
Epoch 1, Iteration 788, loss 1.

Epoch 1, Iteration 950, loss 0.202, batch accuracy 0.900
Epoch 1, Iteration 951, loss 0.360, batch accuracy 0.900
Epoch 1, Iteration 952, loss 0.368, batch accuracy 0.900
Epoch 1, Iteration 953, loss 0.005, batch accuracy 1.000
Epoch 1, Iteration 954, loss 0.386, batch accuracy 0.900
Epoch 1, Iteration 955, loss 0.118, batch accuracy 0.900
Epoch 1, Iteration 956, loss 0.043, batch accuracy 1.000
Epoch 1, Iteration 957, loss 0.024, batch accuracy 1.000
Epoch 1, Iteration 958, loss 0.226, batch accuracy 0.900
Epoch 1, Iteration 959, loss 0.295, batch accuracy 0.800
Epoch 1, Iteration 960, loss 0.087, batch accuracy 1.000
Epoch 1, Iteration 961, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 962, loss 0.468, batch accuracy 0.800
Epoch 1, Iteration 963, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 964, loss 0.125, batch accuracy 0.900
Epoch 1, Iteration 965, loss 0.104, batch accuracy 1.000
Epoch 1, Iteration 966, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 967, loss 0.

Epoch 1, Iteration 1143, loss 0.021, batch accuracy 1.000
Epoch 1, Iteration 1144, loss 0.294, batch accuracy 0.900
Epoch 1, Iteration 1145, loss 0.168, batch accuracy 0.900
Epoch 1, Iteration 1146, loss 0.000, batch accuracy 1.000
Epoch 1, Iteration 1147, loss 0.006, batch accuracy 1.000
Epoch 1, Iteration 1148, loss 0.005, batch accuracy 1.000
Epoch 1, Iteration 1149, loss 0.030, batch accuracy 1.000
Epoch 1, Iteration 1150, loss 0.002, batch accuracy 1.000
Epoch 1, Iteration 1151, loss 0.036, batch accuracy 1.000
Epoch 1, Iteration 1152, loss 0.047, batch accuracy 1.000
Epoch 1, Iteration 1153, loss 0.309, batch accuracy 0.900
Epoch 1, Iteration 1154, loss 0.003, batch accuracy 1.000
Epoch 1, Iteration 1155, loss 0.069, batch accuracy 1.000
Epoch 1, Iteration 1156, loss 0.072, batch accuracy 1.000
Epoch 1, Iteration 1157, loss 0.045, batch accuracy 1.000
Epoch 1, Iteration 1158, loss 0.547, batch accuracy 0.900
Epoch 1, Iteration 1159, loss 0.050, batch accuracy 1.000
Epoch 1, Itera

Epoch 1, Iteration 1344, loss 0.017, batch accuracy 1.000
Epoch 1, Iteration 1345, loss 0.113, batch accuracy 0.900
Epoch 1, Iteration 1346, loss 0.016, batch accuracy 1.000
Epoch 1, Iteration 1347, loss 0.166, batch accuracy 0.900
Epoch 1, Iteration 1348, loss 0.032, batch accuracy 1.000
Epoch 1, Iteration 1349, loss 0.073, batch accuracy 1.000
Epoch 1, Iteration 1350, loss 0.528, batch accuracy 0.800
Epoch 1, Iteration 1351, loss 1.431, batch accuracy 0.800
Epoch 1, Iteration 1352, loss 0.729, batch accuracy 0.800
Epoch 1, Iteration 1353, loss 0.206, batch accuracy 0.800
Epoch 1, Iteration 1354, loss 0.142, batch accuracy 0.900
Epoch 1, Iteration 1355, loss 0.177, batch accuracy 0.900
Epoch 1, Iteration 1356, loss 0.271, batch accuracy 0.800
Epoch 1, Iteration 1357, loss 0.020, batch accuracy 1.000
Epoch 1, Iteration 1358, loss 0.292, batch accuracy 0.900
Epoch 1, Iteration 1359, loss 0.001, batch accuracy 1.000
Epoch 1, Iteration 1360, loss 0.151, batch accuracy 0.900
Epoch 1, Itera

KeyboardInterrupt: 